# AI Model Gateway Demo - Deployment Notebook

This notebook orchestrates the full deployment of the AI Model Gateway infrastructure and agents.

## Steps:
1. Deploy `infra/hub` - API Management and Azure OpenAI resources
2. Deploy `infra/spoke` - AI Foundry project with Model Gateway connection
3. Generate `.env` file from Terraform outputs
4. Deploy and test the agent

## Configuration

In [8]:
# Install required packages (skip if already installed)
%pip install -q "azure-ai-projects>=2.0.0b3" "azure-identity>=1.25.1"

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import os
import subprocess

LOCATION = "swedencentral"
HUB_DIR = "infra/hub"
SPOKE_DIR = "infra/spoke"
os.environ["ARM_SUBSCRIPTION_ID"] = ""
os.environ["ARM_TENANT_ID"] = ""
os.environ["AZURE_TENANT_ID"] = ""

## Helper Functions

In [10]:
def run_terraform(working_dir: str, command: list[str], capture_output: bool = False) -> subprocess.CompletedProcess:
    """Run a terraform command in the specified directory."""
    full_command = ["terraform"] + command
    print(f"📂 {working_dir}")
    print(f"🔧 Running: {' '.join(full_command)}")

    result = subprocess.run(
        full_command,
        cwd=working_dir,
        capture_output=capture_output,
        text=True
    )

    if result.returncode != 0 and not capture_output:
        raise Exception(f"Terraform command failed with exit code {result.returncode}")

    return result


def get_terraform_output(working_dir: str) -> dict:
    """Get terraform outputs as a dictionary."""
    result = run_terraform(working_dir, ["output", "-json"], capture_output=True)
    if result.returncode != 0:
        raise Exception(f"Failed to get outputs: {result.stderr}")

    outputs = json.loads(result.stdout)
    # Extract just the values
    return {k: v["value"] for k, v in outputs.items()}

## Step 1: Deploy Hub Infrastructure

This deploys:
- Azure API Management
- Azure OpenAI with model deployments
- Application Insights

In [11]:
# Initialize Hub
run_terraform(HUB_DIR, ["init", "-upgrade"])

📂 infra/hub
🔧 Running: terraform init -upgrade
Initializing the backend...


Initializing provider plugins...
- Finding azure/azapi versions matching "~> 2.0"...
- Finding hashicorp/random versions matching "~> 3.0"...
- Finding hashicorp/azurerm versions matching "~> 4.0"...
- Using previously-installed azure/azapi v2.8.0
- Using previously-installed hashicorp/random v3.8.0
- Using previously-installed hashicorp/azurerm v4.57.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


CompletedProcess(args=['terraform', 'init', '-upgrade'], returncode=0)

In [12]:
# Apply Hub deployment
run_terraform(HUB_DIR, [
    "apply",
    "-auto-approve",
    f"-var=location={LOCATION}"
])

📂 infra/hub
🔧 Running: terraform apply -auto-approve -var=location=swedencentral


random_string.suffix: Refreshing state... [id=a8w10w]
azapi_resource.app_insights_connection: Refreshing state... [id=/subscriptions/1c8a30db-49c5-45f2-a0e4-63295d5f6324/resourceGroups/ai-model-gateway-demo/providers/Microsoft.CognitiveServices/accounts/foundry-hub-a8w10w/projects/models-hub/connections/app-insights]
data.azurerm_client_config.current: Reading...
azurerm_resource_group.this: Refreshing state... [id=/subscriptions/1c8a30db-49c5-45f2-a0e4-63295d5f6324/resourceGroups/ai-model-gateway-demo]
azurerm_api_management_logger.apim: Refreshing state... [id=/subscriptions/1c8a30db-49c5-45f2-a0e4-63295d5f6324/resourceGroups/ai-model-gateway-demo/providers/Microsoft.ApiManagement/service/model-gateway-a8w10w/loggers/apim-logger]
azurerm_application_insights.this: Refreshing state... [id=/subscriptions/1c8a30db-49c5-45f2-a0e4-63295d5f6324/resourceGroups/ai-model-gateway-demo/providers/Microsoft.Insights/components/insights-a8w10w]
azurerm_log_analytics_workspace.this: Refreshing stat

CompletedProcess(args=['terraform', 'apply', '-auto-approve', '-var=location=swedencentral'], returncode=0)

In [13]:
# Get Hub outputs
hub_outputs = get_terraform_output(HUB_DIR)
print("\n✅ Hub outputs:")
for key, value in hub_outputs.items():
    if "key" in key.lower():
        print(f"   {key}: ***REDACTED***")
    else:
        print(f"   {key}: {value}")

📂 infra/hub
🔧 Running: terraform output -json

✅ Hub outputs:
   apim_gateway_url: https://model-gateway-a8w10w.azure-api.net
   apim_subscription_key: ***REDACTED***
   azure_openai_endpoint: https://model-gateway-a8w10w.azure-api.net/openai
   model_gateway_metadata: {'deploymentInPath': True, 'inferenceAPIVersion': '2024-10-21', 'models': [{'name': 'gpt-4o-mini', 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-4o-mini', 'version': '2024-07-18'}}}, {'name': 'text-embedding-ada-002', 'properties': {'model': {'format': 'OpenAI', 'name': 'text-embedding-ada-002', 'version': '2'}}}]}
   resource_group_name: ai-model-gateway-demo


## Step 2: Deploy Spoke Infrastructure

This deploys:
- AI Foundry Account and Project
- Model Gateway connection to APIM

In [14]:
# Initialize Spoke
run_terraform(SPOKE_DIR, ["init", "-upgrade"])

📂 infra/spoke
🔧 Running: terraform init -upgrade
Initializing the backend...
Initializing provider plugins...
- Finding azure/azapi versions matching "~> 2.0"...
- Finding hashicorp/azurerm versions matching "~> 4.0"...
- Finding latest version of hashicorp/random...
- Using previously-installed hashicorp/random v3.8.0
- Using previously-installed azure/azapi v2.8.0
- Using previously-installed hashicorp/azurerm v4.57.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


CompletedProcess(args=['terraform', 'init', '-upgrade'], returncode=0)

In [15]:
# Prepare spoke variables from hub outputs
model_gateway_var = json.dumps({
    "url": hub_outputs["azure_openai_endpoint"],
    "api_key": hub_outputs["apim_subscription_key"],
    "metadata": hub_outputs["model_gateway_metadata"]
})

# Apply Spoke deployment
run_terraform(SPOKE_DIR, [
    "apply",
    "-auto-approve",
    f"-var=resource_group_name={hub_outputs['resource_group_name']}",
    f"-var=location={LOCATION}",
    f"-var=model_gateway={model_gateway_var}"
])

📂 infra/spoke
🔧 Running: terraform apply -auto-approve -var=resource_group_name=ai-model-gateway-demo -var=location=swedencentral -var=model_gateway={"url": "https://model-gateway-a8w10w.azure-api.net/openai", "api_key": "c3ea48b4301f43c0b0f4346677ecaf89", "metadata": {"deploymentInPath": true, "inferenceAPIVersion": "2024-10-21", "models": [{"name": "gpt-4o-mini", "properties": {"model": {"format": "OpenAI", "name": "gpt-4o-mini", "version": "2024-07-18"}}}, {"name": "text-embedding-ada-002", "properties": {"model": {"format": "OpenAI", "name": "text-embedding-ada-002", "version": "2"}}}]}}
random_string.suffix: Refreshing state... [id=fcdkn7]
data.azurerm_client_config.current: Reading...
azurerm_cognitive_account.this: Refreshing state... [id=/subscriptions/1c8a30db-49c5-45f2-a0e4-63295d5f6324/resourceGroups/ai-model-gateway-demo/providers/Microsoft.CognitiveServices/accounts/foundry-spoke-fcdkn7]
data.azurerm_client_config.current: Read complete after 0s [id=Y2xpZW50Q29uZmlncy9jbGl

CompletedProcess(args=['terraform', 'apply', '-auto-approve', '-var=resource_group_name=ai-model-gateway-demo', '-var=location=swedencentral', '-var=model_gateway={"url": "https://model-gateway-a8w10w.azure-api.net/openai", "api_key": "c3ea48b4301f43c0b0f4346677ecaf89", "metadata": {"deploymentInPath": true, "inferenceAPIVersion": "2024-10-21", "models": [{"name": "gpt-4o-mini", "properties": {"model": {"format": "OpenAI", "name": "gpt-4o-mini", "version": "2024-07-18"}}}, {"name": "text-embedding-ada-002", "properties": {"model": {"format": "OpenAI", "name": "text-embedding-ada-002", "version": "2"}}}]}}'], returncode=0)

In [16]:
# Get Spoke outputs
spoke_outputs = get_terraform_output(SPOKE_DIR)
print("\n✅ Spoke outputs:")
for key, value in spoke_outputs.items():
    print(f"   {key}: {value}")

📂 infra/spoke
🔧 Running: terraform output -json

✅ Spoke outputs:
   cognitive_account_endpoint: https://foundry-spoke-fcdkn7.cognitiveservices.azure.com/
   cognitive_account_name: foundry-spoke-fcdkn7
   project_endpoint: https://foundry-spoke-fcdkn7.services.ai.azure.com/api/projects/my-project
   project_name: my-project


## Step 3: Generate .env File

Create the `.env` file required by `deploy_agent.py` and `test_agent.py`

In [17]:
# Generate .env content
env_content = f"""# Auto-generated by deployment notebook - do not edit manually

# Azure Configuration
AZURE_SUBSCRIPTION_ID={os.environ['ARM_SUBSCRIPTION_ID']}
AZURE_RESOURCE_GROUP={hub_outputs['resource_group_name']}

# AI Foundry Configuration
AZURE_AI_ACCOUNT_NAME={spoke_outputs['cognitive_account_name']}
AZURE_AI_PROJECT_NAME={spoke_outputs['project_name']}
AZURE_AI_PROJECT_ENDPOINT={spoke_outputs['project_endpoint']}

# Agent Configuration
AGENT_NAME=APIMGatewayAgent
AGENT_MODEL=model-gateway/gpt-4o-mini
"""

# Write .env file
with open(".env", "w") as f:
    f.write(env_content)

print("✅ Generated .env file:")
print("-" * 40)
# Print without sensitive data
for line in env_content.split("\n"):
    if line and not line.startswith("#"):
        print(f"   {line}")

✅ Generated .env file:
----------------------------------------
   AZURE_SUBSCRIPTION_ID=1c8a30db-49c5-45f2-a0e4-63295d5f6324
   AZURE_RESOURCE_GROUP=ai-model-gateway-demo
   AZURE_AI_ACCOUNT_NAME=foundry-spoke-fcdkn7
   AZURE_AI_PROJECT_NAME=my-project
   AZURE_AI_PROJECT_ENDPOINT=https://foundry-spoke-fcdkn7.services.ai.azure.com/api/projects/my-project
   AGENT_NAME=APIMGatewayAgent
   AGENT_MODEL=model-gateway/gpt-4o-mini


## Step 4: Deploy Agent

Run `deploy_agent.py` to create the agent in AI Foundry

In [18]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity import DefaultAzureCredential

# Agent configuration
AGENT_NAME = "APIMGatewayAgent"
AGENT_MODEL = "model-gateway/gpt-4o-mini"

print("🚀 Deploying Agent via AI Gateway")
print("=" * 60)

# Initialize the project client
print("\n📡 Connecting to Azure AI Foundry project...")
print(f"   Project: {spoke_outputs['project_name']}")
print(f"   Endpoint: {spoke_outputs['project_endpoint']}")

project_client = AIProjectClient(
    endpoint=spoke_outputs['project_endpoint'],
    credential=DefaultAzureCredential()
)
print("✅ Connected successfully")

# Create an agent using the AI Gateway connection
print("\n🤖 Creating agent with AI Gateway connection...")
print(f"   Model: {AGENT_MODEL}")
print("   (This routes through APIM to Azure OpenAI)")

with project_client:
    agent = project_client.agents.create_version(
        agent_name=AGENT_NAME,
        definition=PromptAgentDefinition(
            model=AGENT_MODEL,
            instructions="You are a helpful AI assistant deployed via Azure API Management. "
            "All your requests are routed through APIM for monitoring and governance.",
        ),
    )

print("\n" + "=" * 60)
print("🎉 Agent Deployment Complete!")
print("=" * 60)
print("\n📋 Agent Details:")
print(f"   Agent Name: {agent.name}")
print(f"   Agent Version: {agent.version}")
print(f"   Model: {AGENT_MODEL}")

🚀 Deploying Agent via AI Gateway

📡 Connecting to Azure AI Foundry project...
   Project: my-project
   Endpoint: https://foundry-spoke-fcdkn7.services.ai.azure.com/api/projects/my-project
✅ Connected successfully

🤖 Creating agent with AI Gateway connection...
   Model: model-gateway/gpt-4o-mini
   (This routes through APIM to Azure OpenAI)

🎉 Agent Deployment Complete!

📋 Agent Details:
   Agent Name: APIMGatewayAgent
   Agent Version: 1
   Model: model-gateway/gpt-4o-mini


## Step 5: Test Agent

Run `test_agent.py` to verify the agent works through the APIM gateway

In [19]:
print("🧪 Testing APIM Gateway Prompt Agent")
print("=" * 60)

project_client = AIProjectClient(
    endpoint=spoke_outputs['project_endpoint'],
    credential=DefaultAzureCredential()
)

with project_client:
    print(f"\n📋 Using Agent: {agent.name} (version {agent.version})")
    print(f"   Model: {AGENT_MODEL}")
    print("   (Routes through APIM)")

    # Test message
    test_message = "Hello! Please tell me a short joke about AI and API gateways."
    print(f"\n💬 User: {test_message}")

    print("\n🔄 Processing through APIM Gateway...")

    # Use the OpenAI client and Conversations/Responses API
    with project_client.get_openai_client() as openai_client:
        # Create a conversation with initial user message
        conversation = openai_client.conversations.create(
            items=[{"type": "message", "role": "user", "content": test_message}],
        )
        print(f"   Conversation created: {conversation.id}")

        # Get response from agent
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            input="",
        )

        print("\n🤖 Assistant Response:")
        print(f"   {response.output_text}")

print("\n" + "=" * 60)
print("✅ Test Complete!")
print("=" * 60)
print("\n📊 Request Flow:")
print("   1. Python SDK → Azure AI Foundry Project")
print("   2. Foundry → model-gateway connection")
print(f"   3. Connection → APIM ({hub_outputs['apim_gateway_url']})")
print("   4. APIM → Azure OpenAI (via managed identity)")
print("   5. Response flows back through APIM (with monitoring/governance)")

🧪 Testing APIM Gateway Prompt Agent

📋 Using Agent: APIMGatewayAgent (version 1)
   Model: model-gateway/gpt-4o-mini
   (Routes through APIM)

💬 User: Hello! Please tell me a short joke about AI and API gateways.

🔄 Processing through APIM Gateway...
   Conversation created: conv_2bd570c9e8b95e0e0066nXFBRBuCm2FUlH9jrpXoyPhhmsxoai

🤖 Assistant Response:
   Why did the AI break up with the API gateway?

Because it couldn’t handle all the requests for attention!

✅ Test Complete!

📊 Request Flow:
   1. Python SDK → Azure AI Foundry Project
   2. Foundry → model-gateway connection
   3. Connection → APIM (https://model-gateway-a8w10w.azure-api.net)
   4. APIM → Azure OpenAI (via managed identity)
   5. Response flows back through APIM (with monitoring/governance)


## Cleanup (Optional)

Destroy all infrastructure when done testing

In [20]:
# ⚠️ DANGER: Uncomment to destroy all resources
# print("🗑️ Destroying Spoke...")
# run_terraform(SPOKE_DIR, ["destroy", "-auto-approve",
#     f"-var=resource_group_name={hub_outputs['resource_group_name']}",
#     f"-var=location={LOCATION}",
#     f"-var=model_gateway={model_gateway_var}"
# ])

# print("🗑️ Destroying Hub...")
# run_terraform(HUB_DIR, ["destroy", "-auto-approve",
#     f"-var=location={LOCATION}"
# ])

# print("✅ All resources destroyed")